<a href="https://colab.research.google.com/github/Bprasad3126/FMML_23B21A4573/blob/main/FMML_M1L3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation

FMML Module 1, Lab 3

In this lab, we will see how augmentation of data samples help in improving the machine learning performance. Augmentation is the process of creating new data samples by making reasonable modifications to the original data samples. This is particularly useful when the size of the training data is small. We will use the MNISt dataset for this lab. We will also reuse functions from the previous labs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn.utils.extmath import cartesian
from skimage.transform import rotate, AffineTransform, warp

rng = np.random.default_rng(seed=42)

In [ ]:
# loading the dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# normalizing the data
train_X = train_X / 255
test_X = test_X / 255

# subsample from images and labels. Otherwise it will take too long!
train_X = train_X[::1200, :, :].copy()
train_y = train_y[::1200].copy()

Let us borrow a few functions from the previous labs:

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    traindata = traindata.reshape(-1, 28*28)
    testdata = testdata.reshape(-1, 28*28)
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel


def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

In this lab, we will use the image pixels themselves as features, instead of extracting features. Each image has 28*28 pixels, so we will flatten them to 784 pixels to use as features. Note that this is very compute intensive and will take a long time. Let us first check the baseline accuracy on the test set without any augmentations. We hope that adding augmentations will help us to get better results.

In [ ]:
testpred = NN(train_X, train_y, test_X)
print("Baseline accuracy without augmentation:",
      Accuracy(test_y, testpred)*100, "%")

Let us try to improve this accuracy using augmentations. When we create augmentations, we have to make sure that the changes reflect what will naturally occur in the dataset. For example, we should not add colour to our samples as an augmentation because they do not naturally occur. We should not also flip the images in MNIST, because flipped images have different meanings for digits. So, we will use the following augmentations:

### Augmentation 1: Rotation

Let us try rotating the image a little. We will use the `rotate` function from the `skimage` module. We will rotate the image by 10 degrees and -10 degrees. Rotation is a reasonable augmentation because the digit will still be recognizable even after rotation and is representative of the dataset.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

axs[0].imshow(train_X[2], cmap="gray")
axs[0].set_title("Original Image")

axs[1].imshow(rotate(train_X[2], 10), cmap="gray")
axs[1].set_title("Rotate +10 degrees")

axs[2].imshow(rotate(train_X[2], -10), cmap="gray")
axs[2].set_title("Rotate -10 degrees")

plt.show()

After rotating, the the class of the image is still the same. Let us make a function to rotate multiple images by random angles. We want a slightly different image every time we run this function. So, we generate a random number between 0 and 1 and change it so that it lies between -constraint/2 and +constraint/2

In [ ]:
def augRotate(sample, angleconstraint):
    """
    This function takes in a sample and an angle constraint and returns the augmented sample
    by rotating the sample by a random angle within the angle constraint

    sample: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    angleconstraint: the maximum angle by which the sample can be rotated

    returns: the augmented sample which is the input sample rotated by a random angle within the angle constraint
    """
    if angleconstraint == 0:
        return sample
    if len(sample.shape) == 2:
        # make sure the sample is 3 dimensional
        sample = np.expand_dims(sample, 0)
    angle = rng.random(len(sample))  # generate random numbers for angles
    # make the random angle constrained
    angle = (angle - 0.5) * angleconstraint
    nsample = sample.copy()  # preallocate the augmented array to make it faster
    for ii in range(len(sample)):
        nsample[ii] = rotate(sample[ii], angle[ii])
    return np.squeeze(nsample)  # take care if the input had only one sample.

This function returns a slightly different image each time we call it. So we can increase the number of images in the sample by any multiple.

In [ ]:
sample = train_X[20]
angleconstraint = 70

fig, axs = plt.subplots(1, 5, figsize=(15, 5))

axs[0].imshow(sample, cmap="gray")
axs[0].set_title("Original Image")

axs[1].imshow(augRotate(sample, angleconstraint), cmap="gray")
axs[1].set_title("Aug. Sample 1")

axs[2].imshow(augRotate(sample, angleconstraint), cmap="gray")
axs[2].set_title("Aug. Sample 2")

axs[3].imshow(augRotate(sample, angleconstraint), cmap="gray")
axs[3].set_title("Aug. Sample 3")

axs[4].imshow(augRotate(sample, angleconstraint), cmap="gray")
axs[4].set_title("Aug. Sample 4")

plt.show()

Let us augment the whole dataset and see if this improves the test accuracy

In [ ]:
# hyperparameters
angleconstraint = 60
naugmentations = 5

# augment
augdata = train_X  # we include the original images also in the augmented dataset
auglabel = train_y
for ii in range(naugmentations):
    augdata = np.concatenate(
        (augdata, augRotate(train_X, angleconstraint))
    )  # concatenate the augmented data to the set
    auglabel = np.concatenate(
        (auglabel, train_y)
    )  # the labels don't change when we augment

# check the test accuracy
testpred = NN(augdata, auglabel, test_X)
print("Accuracy after rotation augmentation:", Accuracy(test_y, testpred)*100, "%")

We can notice a 3-4% improvement compared to non-augmented version of the dataset!

The angle constraint is a hyperparameter which we have to tune using a validation set. (Here we are not doing that for time constraints). Let us try a grid search to find the best angle constraint. We will try angles between 0 and 90 degrees. We can also try different multiples of the original dataset. We will use the best hyperparameters to train the model and check the accuracy on the test set.

In [ ]:
angleconstraints = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]  # the values we want to test
accuracies = np.zeros(
    len(angleconstraints), dtype=float
)  # we will save the values here

for ii in range(len(angleconstraints)):
    # create the augmented dataset
    augdata = train_X  # we include the original images also in the augmented dataset
    auglabel = train_y
    for jj in range(naugmentations):
        augdata = np.concatenate(
            (augdata, augRotate(train_X, angleconstraints[ii]))
        )  # concatenate the augmented data to the set
        auglabel = np.concatenate(
            (auglabel, train_y)
        )  # the labels don't change when we augment

    # check the test accuracy
    testpred = NN(augdata, auglabel, test_X)
    accuracies[ii] = Accuracy(test_y, testpred)
    print(
        "Accuracy after rotation augmentation constrained by",
        angleconstraints[ii],
        "degrees is",
        accuracies[ii]*100,
        "%",
        flush=True,
    )

Let us see the best value for angle constraint: (Ideally this should be done on validation set, not test set)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
# plot the variation of accuracy
ax.plot(angleconstraints, accuracies)
ax.set_xlabel("angle")
ax.set_ylabel("accuracy")

# plot the maximum accuracy
maxind = np.argmax(accuracies)
plt.scatter(angleconstraints[maxind], accuracies[maxind], c="red")

### Augmentation 2: Shear


Let us try one more augmentation: shear. Shear is the transformation of an image in which the x-coordinate of all points is shifted by an amount proportional to the y-coordinate of the point. We will use the `AffineTransform` function from the `skimage` module to shear the image by a small amount between two numbers. We will use the same naive grid search method to find the best hyperparameters for shear. We will use the best hyperparameters to train the model and check the accuracy on the test set.

In [ ]:
def shear(sample, amount):
    """
    This function takes in a sample and an amount and returns the augmented sample
    by shearing the sample by the given amount

    sample: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    amount: the amount by which the sample should be sheared

    returns: the augmented sample which is the input sample sheared by the given amount
    """
    tform = AffineTransform(shear=amount)
    img = warp(sample, tform)

    # Applying shear makes the digit off-center
    # Since all images are centralized, we will do the same here
    col = img.sum(0).nonzero()[0]
    row = img.sum(1).nonzero()[0]
    if len(col) > 0 and len(row) > 0:
        xshift = int(sample.shape[0] / 2 - (row[0] + row[-1]) / 2)
        yshift = int(sample.shape[1] / 2 - (col[0] + col[-1]) / 2)
        img = np.roll(img, (xshift, yshift), (0, 1))
    return img

In [ ]:
sample = train_X[2]
fig, axs = plt.subplots(1, 4, figsize=(15, 5))

axs[0].imshow(sample, cmap="gray")
axs[0].set_title("Original Image")

axs[1].imshow(shear(sample, 0.2), cmap="gray")
axs[1].set_title("Amount = 0.2")

axs[2].imshow(shear(sample, 0.4), cmap="gray")
axs[2].set_title("Amount = 0.4")

axs[3].imshow(shear(sample, 0.6), cmap="gray")
axs[3].set_title("Amount = 0.6")

plt.show()

Create an augmentation function which applies a random shear according to the constraint we provide:

In [ ]:
def augShear(sample, shearconstraint):
    """
    This function takes in a sample and a shear constraint and returns the augmented sample
    by shearing the sample by a random amount within the shear constraint

    sample: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    shearconstraint: the maximum shear by which the sample can be sheared

    returns: the augmented sample which is the input sample sheared by a random amount within the shear constraint
    """
    if shearconstraint == 0:
        return sample
    if len(sample.shape) == 2:
        # make sure the sample is 3 dimensional
        sample = np.expand_dims(sample, 0)
    amt = rng.random(len(sample))  # generate random numbers for shear
    amt = (amt - 0.5) * shearconstraint  # make the random shear constrained
    nsample = sample.copy()  # preallocate the augmented array to make it faster
    for ii in range(len(sample)):
        nsample[ii] = shear(sample[ii], amt[ii])
    return np.squeeze(nsample)  # take care if the input had only one sample.

Let us do a grid search to find the best shear constraint.

In [ ]:
shearconstraints = [
    0,
    0.2,
    0.4,
    0.6,
    0.8,
    1.0,
    1.2,
    1.4,
    1.6,
    1.8,
    2.0,
]  # the values we want to test
accuracies = np.zeros(
    len(shearconstraints), dtype=float
)  # we will save the values here

for ii in range(len(shearconstraints)):
    # create the augmented dataset
    augdata = train_X  # we include the original images also in the augmented dataset
    auglabel = train_y
    for jj in range(naugmentations):
        augdata = np.concatenate(
            (augdata, augShear(train_X, shearconstraints[ii]))
        )  # concatenate the augmented data to the set
        auglabel = np.concatenate(
            (auglabel, train_y)
        )  # the labels don't change when we augment

    # check the test accuracy
    testpred = NN(augdata, auglabel, test_X)
    accuracies[ii] = Accuracy(test_y, testpred)
    print(
        "Accuracy after shear augmentation constrained by",
        shearconstraints[ii],
        "is",
        accuracies[ii]*100,
        "%",
        flush=True,
    )

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
# plot the variation of accuracy
ax.plot(shearconstraints, accuracies)
ax.set_xlabel("angle")
ax.set_ylabel("accuracy")

# plot the maximum accuracy
maxind = np.argmax(accuracies)
plt.scatter(shearconstraints[maxind], accuracies[maxind], c="red")

### Augmentation 3: Rotation + Shear



We can do multiple augmentations at the same time. Here is a function to do both shear and rotation to the sample. In this case, we will have two hyperparameters.

In [ ]:
def augRotateShear(sample, angleconstraint, shearconstraint):
    """
    This function takes in a sample, an angle constraint and a shear constraint and returns the augmented sample
    by rotating the sample by a random angle within the angle constraint and shearing the sample by a random amount within the shear constraint

    sample: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    angleconstraint: the maximum angle by which the sample can be rotated
    shearconstraint: the maximum shear by which the sample can be sheared

    returns: the augmented sample which is the input sample rotated by a random angle within the angle constraint and sheared by a random amount within the shear constraint
    """
    if len(sample.shape) == 2:
        # make sure the sample is 3 dimensional
        sample = np.expand_dims(sample, 0)
    amt = rng.random(len(sample))  # generate random numbers for shear
    amt = (amt - 0.5) * shearconstraint  # make the random shear constrained
    angle = rng.random(len(sample))  # generate random numbers for angles
    # make the random angle constrained
    angle = (angle - 0.5) * angleconstraint
    nsample = sample.copy()  # preallocate the augmented array to make it faster
    for ii in range(len(sample)):
        nsample[ii] = rotate(
            shear(sample[ii], amt[ii]), angle[ii]
        )  # first apply shear, then rotate
    return np.squeeze(nsample)  # take care if the input had only one sample.

Since we have two hyperparameters, we have to do the grid search on a 2 dimensional matrix. We can use our previous experience to inform where to search for the best hyperparameters.

In [ ]:
shearconstraints = [
    0,
    0.2,
    0.4,
    0.6,
    0.8,
    1.0,
    1.2,
    1.4,
    1.6,
]  # the values we want to test
angleconstraints = [0, 10, 20, 30, 40, 50, 60]  # the values we want to test
# cartesian product of both
hyp = cartesian((shearconstraints, angleconstraints))

accuracies = np.zeros(len(hyp), dtype=float)  # we will save the values here

for ii in range(len(hyp)):
    # create the augmented dataset
    augdata = train_X  # we include the original images also in the augmented dataset
    auglabel = train_y
    for jj in range(naugmentations):
        augdata = np.concatenate(
            (augdata, augRotateShear(train_X, hyp[ii][0], hyp[ii][1]))
        )  # concatenate the augmented data to the set
        auglabel = np.concatenate(
            (auglabel, train_y)
        )  # the labels don't change when we augment

    # check the test accuracy
    testpred = NN(augdata, auglabel, test_X)
    accuracies[ii] = Accuracy(test_y, testpred)
    print(
        "Accuracy after augmentation shear:",
        hyp[ii][0],
        "angle:",
        hyp[ii][1],
        "is",
        accuracies[ii]*100,
        "%",
        flush=True,
    )

Accuracy after augmentation shear: 0.0 angle: 0.0 is 63.32 %
Accuracy after augmentation shear: 0.0 angle: 10.0 is 63.959999999999994 %
Accuracy after augmentation shear: 0.0 angle: 20.0 is 60.64000000000001 %
Accuracy after augmentation shear: 0.0 angle: 30.0 is 63.019999999999996 %
Accuracy after augmentation shear: 0.0 angle: 40.0 is 64.14999999999999 %
Accuracy after augmentation shear: 0.0 angle: 50.0 is 61.72 %
Accuracy after augmentation shear: 0.0 angle: 60.0 is 63.7 %
Accuracy after augmentation shear: 0.2 angle: 0.0 is 63.41 %
Accuracy after augmentation shear: 0.2 angle: 10.0 is 61.25000000000001 %
Accuracy after augmentation shear: 0.2 angle: 20.0 is 60.6 %
Accuracy after augmentation shear: 0.2 angle: 30.0 is 60.07 %
Accuracy after augmentation shear: 0.2 angle: 40.0 is 63.690000000000005 %
Accuracy after augmentation shear: 0.2 angle: 50.0 is 60.12 %
Accuracy after augmentation shear: 0.2 angle: 60.0 is 63.72 %
Accuracy after augmentation shear: 0.4 angle: 0.0 is 63.37000

Let us plot it two dimensionally to see which is the best value for the hyperparameters:

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
im = ax.imshow(
    accuracies.reshape((len(shearconstraints), len(angleconstraints))), cmap="hot"
)
ax.set_xlabel("Angle")
ax.set_ylabel("Shear")
ax.set_xticks(np.arange(len(angleconstraints)))
ax.set_xticklabels(angleconstraints)
ax.set_yticks(np.arange(len(shearconstraints)))
ax.set_yticklabels(shearconstraints)
plt.colorbar(im)

It seems that rotation and shear don't mix! The best accuracy is when rotation is zero.

## Questions
Try these questions for better understanding. You may not be able to solve all of them.
1. What is the best value for angle constraint and shear constraint you got? How much did the accuracy improve as compared to not using augmentations?
2. Can you increase the accuracy by increasing the number of augmentations from each sample?
3. Try implementing a few augmentations of your own and experimenting with them. A good reference is <a href=https://www.analyticsvidhya.com/blog/2019/12/image-augmentation-deep-learning-pytorch/>here. </a>
4. Try combining various augmentations. What is the highest accuracy you can get? What is the smallest training dataset you can take and still get accuracy above 50%?

Whenever you do any experiment, a good practice is to vary the hyperparameters gradually and create a graph of your results, like we did for gridsearch.

It sounds like you are referring to data augmentation techniques that involve transformations like angle constraint (rotation) and shear constraint (shearing) during model training, which are commonly used in computer vision tasks. These augmentations can help increase the robustness of a model by making it invariant to these transformations, but the "best" values for these constraints are often task-dependent.

Here’s how you might approach this:

Angle Constraint (Rotation):

Typical ranges for rotation might be between -30° to +30°, depending on the problem. If the data has significant orientation variations, a wider range may be suitable.
If objects in your dataset tend to appear in a fixed orientation (e.g., faces), you might restrict this to smaller values, such as -10° to +10°.
Shear Constraint:

Shearing usually works best within a small range, often between -0.2 and +0.2 radians. Larger shear angles can distort objects excessively.
For datasets where perspective changes are important (e.g., documents or vehicles in motion), a higher shear might be reasonable.
Measuring Improvement:
You would typically evaluate the effectiveness of these augmentations by comparing the validation/test accuracy of models trained with and without these constraints.

Without augmentation: You train the model on the raw data and evaluate the performance.

With augmentation: You apply the angle and shear transformations during training and check if the model generalizes better to unseen data.

Example Results (Hypothetical):
Without augmentation: Baseline accuracy is 85%.
With angle constraint (-10° to +10°) and shear constraint (-0.2 to +0.2): After applying augmentations, accuracy improves to 88%.
Thus, the augmentation could potentially lead to a 3% improvement in accuracy.

The actual improvement depends on the dataset and model architecture. Have you experimented with these parameters already or are you planning to try this out?

Yes, increasing the number of augmentations from each sample can indeed improve model performance, particularly if you have a limited dataset. Here's how this works and why it can help:

How Increasing Augmentations Helps:
More Diverse Training Data:

By applying more augmentations (rotations, shears, flips, etc.) to each sample, you effectively create a larger and more diverse training set.
This forces the model to learn features that are invariant to those transformations (e.g., orientation, scale, brightness), making it more robust to variations in unseen data.
Regularization Effect:

More augmented samples act as a form of regularization, helping the model avoid overfitting to the specific examples in the training data.
The model is less likely to memorize the dataset and more likely to learn generalized patterns.
Reducing Data Scarcity:

In cases where you have limited labeled data, augmenting each sample multiple times can simulate a larger dataset, which is particularly useful for deep learning models that are data-hungry.
To What Extent Should You Augment?
Too Few Augmentations: The model might still overfit and not generalize well to unseen data.
Too Many Augmentations: Excessive augmentation, especially with aggressive transformations, might distort the data to the point where it no longer represents the underlying distribution, which could hurt performance.
Optimal Number of Augmentations:
In practice, generating 5 to 10 augmentations per sample is often a good starting point, but the optimal number depends on the dataset, task, and model.
Experiment Results (Hypothetical):
No augmentations: 85% accuracy.
1 augmentation per sample: 87% accuracy.
5 augmentations per sample: 90% accuracy.
10 augmentations per sample: 91% accuracy (at this point, the gains may start diminishing).
So, yes, increasing the number of augmentations from each sample can improve accuracy up to a certain point. Beyond that, the improvements may be marginal.

Do you have a specific dataset in mind where you're planning to try this?

It sounds like you're looking for practical experimentation with data augmentation techniques, possibly for a vision-related task. Since I can't directly access external sources or implement code on external data here, I can guide you on how to proceed and help with code snippets to apply augmentations like rotation, shear, and others using libraries like TensorFlow/Keras or PyTorch.

Sample Augmentation Techniques:
Here are a few augmentations you can easily implement:

Rotation (e.g., rotate between -30° to +30°)
Shearing (e.g., horizontal or vertical shear)
Zooming (random zoom in or out of the image)
Flipping (horizontal or vertical flips)
Color Adjustments (brightness, contrast, saturation, etc.)
If you're using TensorFlow/Keras, you can apply these using ImageDataGenerator or tf.image functions. Here's an example:

Example using TensorFlow/Keras:
python
Copy code
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# Sample image for demonstration
image_path = 'path_to_your_image.jpg'
image = tf.keras.preprocessing.image.load_img(image_path)
image = tf.keras.preprocessing.image.img_to_array(image)
image = np.expand_dims(image, axis=0)

# Define augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=30,  # Rotate images up to 30 degrees
    shear_range=0.2,    # Shear transformation
    zoom_range=0.2,     # Random zoom
    horizontal_flip=True,  # Flip horizontally
    brightness_range=[0.8, 1.2],  # Random brightness
)

# Apply augmentations
augmented_images = datagen.flow(image, batch_size=1)

# Plot original and augmented images
plt.figure(figsize=(12, 6))

# Original image
plt.subplot(1, 5, 1)
plt.imshow(image[0].astype('uint8'))
plt.title('Original Image')

# Display 4 augmented images
for i in range(4):
    plt.subplot(1, 5, i+2)
    aug_img = augmented_images.next()[0].astype('uint8')
    plt.imshow(aug_img)
    plt.title('Augmented Image {}'.format(i+1))

plt.show()
Example using PyTorch:
If you're using PyTorch, here's how you can implement similar augmentations using torchvision.transforms:

python
Copy code
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Load image
image_path = 'path_to_your_image.jpg'
image = Image.open(image_path)

# Define augmentation techniques
augmentations = transforms.Compose([
    transforms.RandomRotation(30),  # Random rotation up to 30 degrees
    transforms.RandomResizedCrop(224),  # Random crop and resize to 224x224
    transforms.RandomHorizontalFlip(),  # Flip horizontally
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Color adjustments
    transforms.ToTensor()  # Convert to Tensor
])

# Apply augmentations to the image
augmented_images = [augmentations(image) for _ in range(4)]

# Plot original and augmented images
plt.figure(figsize=(12, 6))

# Original image
plt.subplot(1, 5, 1)
plt.imshow(image)
plt.title('Original Image')

# Display 4 augmented images
for i in range(4):
    plt.subplot(1, 5, i+2)
    plt.imshow(augmented_images[i].permute(1, 2, 0))  # Convert back to HWC format
    plt.title('Augmented Image {}'.format(i+1))

plt.show()
Experimentation:
Variation of Parameters: You can experiment with different ranges for rotation_range, shear_range, zoom_range, etc.
Number of Augmentations: You can generate multiple augmentations for each sample in your dataset and observe how the model accuracy improves.
If you share more details about your dataset and augmentation objectives, I can assist in refining the code or help with more specific augmentation strategies

Since I can't directly run code or experiments in this environment, I’ll guide you through the process you can use to explore the highest possible accuracy and the smallest dataset size for achieving at least 50% accuracy using various augmentations.

Here's a detailed experiment setup you can implement:

1. Model Selection and Dataset
Choose a simple dataset like CIFAR-10 or MNIST, which are well-known for image classification tasks.

CIFAR-10 consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class.
MNIST consists of 70,000 28x28 grayscale images of digits (0-9).
For the experiment, we'll assume you're using CIFAR-10 as an example.

2. Augmentation Combinations
Using TensorFlow/Keras or PyTorch, apply various combinations of augmentations such as:

Rotation: Randomly rotate the images.
Shear: Apply a horizontal/vertical shear.
Zoom: Apply zoom in/out.
Flip: Flip images horizontally or vertically.
Brightness, Contrast, Saturation Adjustments: Randomly vary these properties.
Cropping/Rescaling: Random cropping and resizing to the original dimension.
3. PyTorch Example: Augmentation Pipeline
Here’s an example of combining augmentations in PyTorch:

python
Copy code
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Define augmentation transformations
augmentations = transforms.Compose([
    transforms.RandomRotation(30),        # Rotate images
    transforms.RandomHorizontalFlip(),    # Flip images horizontally
    transforms.RandomAffine(degrees=15, shear=10),  # Apply shear transformation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Brightness/Contrast adjustment
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),  # Random crop
    transforms.ToTensor()  # Convert to Tensor
])

# Load CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=augmentations)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a simple CNN model (or use a pre-trained model)
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):
    def _init_(self):
        super(SimpleCNN, self)._init_()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*6*6, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")

# Train the model
train(model, train_loader, criterion, optimizer)

# Test the model accuracy
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}%")

# Test the model
test(model, test_loader)
4. Experimentation: Reducing Dataset Size
To test the effect of reducing the dataset size while using augmentations, you can take the following steps:

Step 1: Full Dataset with Augmentations
Train the model on the full CIFAR-10 dataset (50,000 training images) with the augmentation pipeline and measure the baseline accuracy. This gives you a good starting point to see how well the model performs with augmentation.

Step 2: Reduce the Dataset to 50%
Randomly select 50% of the training data (25,000 images).
Apply the same augmentations.
Train the model and measure accuracy. If the accuracy is still high (e.g., above 50%), you can proceed to reduce the dataset further.
Step 3: Continue Reducing
25% dataset (12,500 images): Test the accuracy.
10% dataset (5,000 images): Test the accuracy.
5% dataset (2,500 images): Test the accuracy.
As you reduce the dataset size, the augmentations should help keep the accuracy above 50% by effectively increasing the diversity of the training data.

Expected Results (Hypothetical)
From similar experiments, here are hypothetical results you might expect (these will depend on the model architecture and dataset used):

Full Dataset (with augmentations): ~85% accuracy.
50% Dataset (with augmentations): ~80% accuracy.
25% Dataset (with augmentations): ~70% accuracy.
10% Dataset (with augmentations): ~55-60% accuracy.
5% Dataset (with augmentations): ~50-55% accuracy.
With careful tuning, you could achieve above 50% accuracy using as little as 5-10% of the dataset with augmentations.

Conclusion
Combining augmentations like rotation, shear, flipping, color jittering, and random cropping helps significantly improve model robustness and performance, especially on small datasets.
Dataset size reduction experiments allow you to find the smallest dataset that achieves a minimum accuracy threshold (50% in this case). Applying augmentations enables you to reduce the dataset size while maintaining reasonable accuracy.
You can implement this pipeline and experiment with your own dataset to find the highest possible accuracy and the smallest dataset size that works for your use case.